# OpenRoad Project

TensorFlow Extended (TXF) - helps build production pipelines for ML

## Set up

### Upgrade Pip

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Install TFX

In [ ]:
!pip install tfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Restart Runtime

### Import Packages

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


check library versions

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.9.1
TFX version: 1.9.1


### Set up pipeline paths

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the Traffic Pipeline example.
_traffic_root = os.path.join(_tfx_root, 'examples/traffic_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/traffic_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO) 

### Download example data

### Download example data
We download the example dataset for use in our TFX pipeline.

With this dataset, we will build a model that predicts the `traffic_count`.

In [ ]:


_data_root = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
DATA_PATH = 'https://drive.google.com/uc?export=view&id=1mp-3fpzcFN8FglVIR19ts82G93Xp1Ogj'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datadr493mf4/data.csv', <http.client.HTTPMessage at 0x7f284d25e790>)

quick look at CSV file

In [ ]:
!head {_data_filepath}

the_geom,cartodb_id,the_geom_webmercator,record_id,station_key,station_id,traffic_direction_seq,traffic_direction_name,cardinal_direction_seq,cardinal_direction_name,classification_seq,classification_type,count_type,year,period,partial_year,latest_date,traffic_count,data_start_date,data_end_date,data_duration,data_availability,data_reliability,data_quality_indicator,publish,md5,updated_on
,181393,,,58875,F3FWY006,2,PRESCRIBED AND COUNTER,9,BOTH,3,HEAVY VEHICLES,TRAFFIC COUNT,2018,PM PEAK,false,,2178,,,,-1,-1,0,1,b5cc8afd807d718fa40f2b7872458136,2018-07-09 03:45:51.087472+00
,181394,,,56980,43246,1,PRESCRIBED,1,NORTH,0,UNCLASSIFIED,TRAFFIC COUNT,2018,PUBLIC HOLIDAYS,false,,18049,,,,23,100,0,1,7a3f1cac5b1418067160da2436693738,2018-07-09 03:45:51.087472+00
,181396,,,25868001,29010,0,COUNTER,5,SOUTH,3,HEAVY VEHICLES,TRAFFIC COUNT,2018,OFF PEAK,false,,2879,,,,24,98,0,1,67dbe0263e4b13b3a061818df4cd6007,2018-07-09 03:45:51.087472+00
,181397,,,58875,F3FWY006,0,COUNTER,5,SOUTH,2,LIGHT VEHICLES,

### Create the InteractveContext

Last, we create an InteractiveContext, which will allow us to run TFX components interactively in this notebook.

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## Run TFX Components interactively

https://www.youtube.com/watch?v=7-VxHnkCJ5Y

In the cells that follow, we create TFX components one-by-one, run each of them, and visualize their output artifacts.

### ExampleGen
The `ExampleGen` component is usually at the start of a TFX pipeline. It will:

1.   Split data into training and evaluation sets (by default, 2/3 training + 1/3 eval)
2.   Convert data into the `tf.Example` format
3.   Copy data into the `_tfx_root` directory for other components to access

### Enabling the Cache
When using the InteractiveContext in a notebook to develop a pipeline you can control when individual components will cache their outputs. Set enable_cache to True when you want to reuse the previous output artifacts that the component generated. Set enable_cache to False when you want to recompute the output artifacts for a component, if you are making changes to the code for example.

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data /tmp/tfx-datadr493mf4/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Let's examine the output artifacts of ExampleGen. This component produces two artifacts, training examples and evaluation examples:

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2022-09-01T14_32_12.662366-mpj56axs/CsvExampleGen/examples/1


We can also take a look at the first three training examples:

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "cardinal_direction_name"
    value {
      bytes_list {
        value: "BOTH"
      }
    }
  }
  feature {
    key: "cardinal_direction_seq"
    value {
      int64_list {
        value: 9
      }
    }
  }
  feature {
    key: "cartodb_id"
    value {
      int64_list {
        value: 181393
      }
    }
  }
  feature {
    key: "classification_seq"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "classification_type"
    value {
      bytes_list {
        value: "HEAVY VEHICLES"
      }
    }
  }
  feature {
    key: "count_type"
    value {
      bytes_list {
        value: "TRAFFIC COUNT"
      }
    }
  }
  feature {
    key: "data_availability"
    value {
      int64_list {
        value: -1
      }
    }
  }
  feature {
    key: "data_duration"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "data_end_date"
    value {
      bytes_list {
      }
    }
  }
  feature {
    key: "data_q

Now that `ExampleGen` has finished ingesting the data, the next step is data analysis.

### StatisticsGen

The StatisticsGen component computes statistics over your dataset for data analysis, as well as for use in downstream components. It uses the TensorFlow Data Validation library.

StatisticsGen takes as input the dataset we just ingested using ExampleGen.

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2022-09-01T14_32_12.662366-mpj56axs/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2022-09-01T14_32_12.662366-mpj56axs/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

After `StatisticsGen` finishes running, we can visualize the outputted statistics.

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

The SchemaGen component generates a schema based on your data statistics. (A schema defines the expected bounds, types, and properties of the features in your dataset.) It also uses the TensorFlow Data Validation library.

Note: The generated schema is best-effort and only tries to infer basic properties of the data. It is expected that you review and modify it as needed.

SchemaGen will take as input the statistics that we generated with StatisticsGen, looking at the training split by default.

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2022-09-01T14_32_12.662366-mpj56axs/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

After `SchemaGen` finishes running, we can visualize the generated schema as a table.

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'cardinal_direction_name',STRING,required,single,'cardinal_direction_name'
'cardinal_direction_seq',INT,required,single,-
'cartodb_id',INT,required,single,-
'classification_seq',INT,required,single,-
'classification_type',STRING,required,single,'classification_type'
'count_type',STRING,required,single,'count_type'
'data_availability',INT,required,single,-
'data_duration',INT,required,,-
'data_end_date',BYTES,required,,-


,Values
Domain,
'cardinal_direction_name',"'BOTH', 'EAST', 'EASTBOUND AND WESTBOUND', 'NORTH', 'NORTHBOUND AND SOUTHBOUND', 'SOUTH', 'WEST'"
'classification_type',"'ALL VEHICLES', 'HEAVY VEHICLES', 'LIGHT VEHICLES', 'UNCLASSIFIED'"
'count_type','TRAFFIC COUNT'
'latest_date',"'1970-01-01 00:00:00+00', '2017-04-30 00:00:00+00', '2017-10-31 00:00:00+00', '2018-02-28 00:00:00+00'"
'period',"'ALL DAYS', 'AM PEAK', 'OFF PEAK', 'PM PEAK', 'PUBLIC HOLIDAYS', 'WEEKDAYS', 'WEEKENDS'"
'traffic_direction_name',"'COUNTER', 'PRESCRIBED', 'PRESCRIBED AND COUNTER'"
'updated_on',"'2018-01-09 22:39:10.806238+00', '2018-04-05 05:43:37.380252+00', '2018-06-12 02:23:24.195348+00', '2018-07-09 03:45:51.087472+00', '2018-08-21 03:52:39.506693+00', '2018-11-21 01:20:04.829912+00', '2018-12-14 05:33:24.64657+00', '2019-01-11 04:54:31.393786+00', '2019-02-05 03:50:09.86113+00', '2019-04-03 00:07:11.241656+00', '2019-05-01 23:43:38.082089+00', '2019-06-03 01:35:24.251276+00', '2019-06-30 23:14:15.441676+00', '2019-07-31 23:03:22.71287+00', '2019-08-22 06:24:54.260377+00', '2019-09-03 00:21:04.342687+00', '2019-10-03 22:59:10.760487+00', '2019-10-31 18:58:26.405026+00', '2019-11-13 02:05:08.419435+00', '2019-12-02 02:48:48.344662+00', '2020-03-30 02:37:08.857586+00', '2020-05-04 01:53:48.001418+00', '2020-06-03 00:53:08.27959+00', '2020-07-03 01:51:38.474591+00', '2020-08-05 02:19:46.791976+00', '2020-09-03 01:24:31.45111+00', '2020-10-09 00:29:46.207129+00', '2020-11-05 02:30:14.582359+00', '2020-12-02 05:32:52.054292+00', '2021-01-11 05:34:57.037603+00', '2021-02-16 05:16:49.389723+00', '2021-03-04 09:57:38.233757+00', '2021-04-04 11:07:58.450257+00', '2021-05-04 11:16:20.995384+00', '2021-06-04 11:25:51.216212+00', '2021-07-04 12:06:06.645394+00', '2021-08-04 11:41:27.927875+00', '2021-09-04 11:43:21.427643+00', '2021-10-04 10:53:09.844988+00', '2021-11-04 11:00:39.03746+00', '2021-12-04 11:06:49.907795+00', '2022-01-04 11:16:30.617401+00', '2022-04-04 10:58:39.510802+00', '2022-05-04 11:05:55.34363+00', '2022-06-04 11:14:03.429682+00', '2022-07-04 11:20:26.944323+00', '2022-08-04 11:26:22.809884+00'"


Each feature in your dataset shows up as a row in the schema table, alongside its properties. The schema also captures all the values that a categorical feature takes on, denoted as its domain.

### ExampleValidator
The `ExampleValidator` component detects anomalies in your data, based on the expectations defined by the schema.

ExampleValidator will take as input the statistics from StatisticsGen, and the schema from SchemaGen.

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2022-09-01T14_32_12.662366-mpj56axs/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2022-09-01T14_32_12.662366-mpj56axs/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

After `ExampleValidator` finishes running, we can visualize the anomalies as a table.

In [ ]:
context.show(example_validator.outputs['anomalies'])

In the anomalies table, we can see that there are no anomalies. This is what we'd expect, since this the first dataset that we've analyzed and the schema is tailored to it. You should review this schema -- anything unexpected means an anomaly in the data. Once reviewed, the schema can be used to guard future data, and anomalies produced here can be used to debug model performance, understand how your data evolves over time, and identify data errors.

### Transform
The `Transform` component performs feature engineering for both training and serving.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module that contains user-defined Transform code.

Note: The `%%writefile` cell magic will save the contents of the cell as a `.py` file on disk. This allows the `Transform` component to load your code as a module.


In [ ]:
_traffic_constants_module_file = 'traffic_constants.py'

In [ ]:
%%writefile {_traffic_constants_module_file}


### Trainer

#### Analyze Training With TensorBoard

### Evaluator

### Pusher